# **Export YOLOv8 model**

## **Initial configuration**

**Install packages**

In [ ]:
# !pip install onnx==1.17.0
# !pip install onnxslim==0.1.34
# !pip install onnxruntime==1.20.1
# !pip install onnxruntime-gpu==1.20.1
# !pip install tensorrt==10.7.0

**Import libraries**

In [10]:
import os
import shutil
import subprocess
import torch
import yaml

from ultralytics import YOLO

In [2]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4034MiB)
Setup complete ✅ (8 CPUs, 7.6 GB RAM, 69.0/294.2 GB disk)


**Graphic cards features**

In [3]:
# Verify cuda available device
cuda_device_num = torch.cuda.device_count()
if not cuda_device_num:
  print("\033[93m WARNING: No cuda devices found. \033[0m")
else:
  print(f"\033[34m INFO: There {'is' if cuda_device_num == 1 else 'are'} {cuda_device_num} pytorch cuda devices. \033[0m")
  print(f"\033[34m       Pyorch cuda version: {torch.version.cuda} \033[0m")
  print(f"\033[34m-\033[0m" * 35)
  for index in  range(cuda_device_num):
    print(f"\033[34m GPU {index}: {torch.cuda.get_device_name(index)} \033[0m")
    print(f"\033[34m \tTotal cuda device memory {torch.cuda.mem_get_info(0)[1] // 2 ** 20} MiB\033[0m")
    print(f"\033[34m \tFree cuda device memory {torch.cuda.mem_get_info(index)[0] // 2 ** 20} MiB\033[0m\n")

  print("Graphics card information")
  print(subprocess.check_output("nvidia-smi").decode())

 INFO: There is 1 pytorch cuda devices. 
       Pyorch cuda version: 12.4 
-----------------------------------
 GPU 0: NVIDIA GeForce GTX 1050 Ti 
 	Total cuda device memory 4034 MiB
 	Free cuda device memory 2222 MiB

Graphics card information
Mon Jan 27 19:54:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti     Off |   00000000:01:00.0 Off |           

**Directory structure**

In [4]:
# Run local or not
PROJ_ROOT = os.getcwd()
local = True if os.path.basename(PROJ_ROOT) == "notebooks" else False

# Directories path
if not local:
  os.makedirs("models/trained", exist_ok=True)
  data_relative_path ="data"
  models_relative_path = "models"
else:
  data_relative_path ="../data/processed"
  models_relative_path = "../models"

TRAINED_MODEL_DIR = os.path.join(PROJ_ROOT, models_relative_path, "trained")
PROCESSED_DATA_DIR = os.path.join(PROJ_ROOT, data_relative_path)

# Print directories paths
print(f"\033[34m INFO: Processed data in {PROCESSED_DATA_DIR}. \033[0m")
print(f"\033[34m INFO: Trained model in {TRAINED_MODEL_DIR}. \033[0m")

# Print warning
print(f"\033[93m WARNING: You need to load the trained model into {TRAINED_MODEL_DIR}. \033[0m")

 INFO: Processed data in /home/harvey/Documents/yolo_ppe_detection/notebooks/../data/processed. 
 INFO: Trained model in /home/harvey/Documents/yolo_ppe_detection/notebooks/../models/trained. 


**Configuration variables**

In [6]:
# Configuration class
class CFG:
  # Dataset settings (Roboflow)
  DATASET_WORKSPACE = "deeplearning-cwudo"
  DATASET_PROJECT = "yolo_ppe_detection"
  DATASET_FORMAT = "yolov8"
  DATASET_VERSION = 5
  DATASET_NAME = f"ppe_dataset_v{DATASET_VERSION}"
  DATASET_PATH = os.path.join(PROCESSED_DATA_DIR, DATASET_NAME)
  DATASET_YAML_PATH = os.path.join(DATASET_PATH, "data.yaml")

  # Export variables
  EXPORT_FORMAT = "engine"
  EXPORT_MODEL_NAME = "ppe_detection.pt"
  EXPORT_DEVICE = "0" if cuda_device_num else "cpu"
  EXPORT_TRAINED_MODEL_PATH = "/home/harvey/Documents/yolo_ppe_detection/models/train_yolov8n_ppe_detection_debug/pt_3_epochs/weights/best.pt"
  EXPORT_MODEL_PATH = os.path.join(TRAINED_MODEL_DIR, EXPORT_MODEL_NAME)

# Print information
print(f"\nDataset information")
print(f"Dataset path: {CFG.DATASET_PATH}")

print("\nExport information")
print(f"Export format: {CFG.EXPORT_FORMAT}")
print(f"Export trained model path: {CFG.EXPORT_TRAINED_MODEL_PATH}")
print(f"Export model path: {CFG.EXPORT_MODEL_PATH}")


Dataset information
Dataset path: /home/harvey/Documents/yolo_ppe_detection/notebooks/../data/processed/ppe_dataset_v5

Export information
Export format: engine
Export trained model path: /home/harvey/Documents/yolo_ppe_detection/models/train_yolov8n_ppe_detection_debug/pt_3_epochs/weights/best.pt
Export model path: /home/harvey/Documents/yolo_ppe_detection/notebooks/../models/trained/ppe_detection.pt


## **Dataset**

**Verify dataset**

In [9]:
# Dataset paths
dataset_path_exist = os.path.isdir(CFG.DATASET_PATH)
dataset_yaml_file_exist = os.path.isfile(CFG.DATASET_YAML_PATH)

# Check if path exists
if not dataset_path_exist or not dataset_yaml_file_exist:
  print("\033[93m WARNING: Check dataset path or download again. \033[0m")
else:
  print("\033[34m INFO: The dataset exists. \033[0m\n")
  # Plot dataset yaml file
  with open(CFG.DATASET_YAML_PATH, "r") as file:
    try:
      data = yaml.safe_load(file)
      yaml_data = yaml.dump(data, default_style=False)
      print(yaml_data)

    except yaml.YAMLError as e:
      print("Error reading YAML: ", e)
  file.close()

 INFO: The dataset exists. 

names:
- hard_hat
- no_hard_hat
- no_safety_harness
- no_safety_vest
- person
- safety_harness
- safety_vest
nc: 7
roboflow:
  license: CC BY 4.0
  project: yolo_ppe_detection
  url: https://universe.roboflow.com/deeplearning-cwudo/yolo_ppe_detection/dataset/5
  version: 5
  workspace: deeplearning-cwudo
test: /home/harvey/Documents/yolo_ppe_detection/notebooks/../data/processed/ppe_dataset_v5/test/images
train: /home/harvey/Documents/yolo_ppe_detection/notebooks/../data/processed/ppe_dataset_v5/train/images
val: /home/harvey/Documents/yolo_ppe_detection/notebooks/../data/processed/ppe_dataset_v5/valid/images



## **Export**

**Copy trained model and verify**

In [11]:
if not os.path.isfile(CFG.EXPORT_TRAINED_MODEL_PATH):
  print(f"\033[31m ERROR: Is posible the trained model {CFG.EXPORT_TRAINED_MODEL_PATH} don't exist. \033[0m")
else:
  shutil.copy(CFG.EXPORT_TRAINED_MODEL_PATH, CFG.EXPORT_MODEL_PATH)
  print(f"\033[34m INFO: The trained model from {CFG.EXPORT_TRAINED_MODEL_PATH} was copied to {CFG.EXPORT_MODEL_PATH}. \033[0m")

 INFO: The trained model from /home/harvey/Documents/yolo_ppe_detection/models/train_yolov8n_ppe_detection_debug/pt_3_epochs/weights/best.pt was copied to /home/harvey/Documents/yolo_ppe_detection/notebooks/../models/trained/ppe_detection.pt. 


**Export settings**

In [12]:
export_args = [
  "format",
  "imgsz",
  "keras",
  "optimize",
  "half",
  "int8",
  "dynamic",
  "simplify",
  "opset",
  "workspace",
  "nms",
  "batch",
  "device",
  "name",
  "data"
]

format_args = {
  "torchscript": [1, 3, 11],
  "onnx": [1, 4, 6, 7, 8, 11],
  "openvino": [1, 4, 5, 6, 11],
  "engine": [1, 4, 5, 6, 7, 9, 11, 14],
  "coreml": [1, 4, 5, 10, 11],
  "saved_model": [1, 2, 5, 11],
  "pb": [1, 11],
  "tflite": [1, 4, 5, 11],
  "edgetpu": [1],
  "tfjs": [1, 4, 5, 11],
  "paddle": [1, 11],
  "mnn": [1, 4, 5, 11],
  "ncnn": [1, 4, 11],
  "imx": [1, 5],
  "rknn": [1, 11, 13]
}

print(f"\033[34m INFO: Export in {CFG.EXPORT_FORMAT} format, change the next arguments. \033[0m")
id_args = format_args[CFG.EXPORT_FORMAT]
for id_arg in id_args:
  print(f"\033[34m\t {export_args[id_arg]} \033[0m")

 INFO: Export in engine format, change the next arguments. 
	 imgsz 
	 half 
	 int8 
	 dynamic 
	 simplify 
	 workspace 
	 batch 
	 data 


In [13]:
cfg_export = {
  "format": CFG.EXPORT_FORMAT,
  "imgsz": 640,
  #"keras": False,
  #"optimize": False,
  "half": False,
  "int8": True,
  "dynamic": True,
  "simplify": True,
  #"opset": None,
  "workspace": 2,
  #"nms": False,
  "batch": 1,
  "device": CFG.EXPORT_DEVICE,
  "name": None,
  "data": CFG.DATASET_YAML_PATH
}

for key, value in cfg_export.items():
  print(key, ":", value)

format : engine
imgsz : 640
half : False
int8 : True
dynamic : True
simplify : True
workspace : 2
batch : 1
device : 0
name : None
data : /home/harvey/Documents/yolo_ppe_detection/notebooks/../data/processed/ppe_dataset_v5/data.yaml


**Load the model**

In [14]:
try:
  export_model = YOLO(CFG.EXPORT_MODEL_PATH)
  print(f"The {CFG.EXPORT_MODEL_PATH} was loaded.")
except:
  print(f"Error loading the {CFG.EXPORT_MODEL_PATH} model.")

The /home/harvey/Documents/yolo_ppe_detection/notebooks/../models/trained/ppe_detection.pt was loaded.


**Export the model**

In [ ]:
export_model.export(**cfg_export)